In [ ]:
!pip install asrtoolkit
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install nltk

import io
import os
from datetime import datetime
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from scipy import stats
from nltk.metrics import distance
import string
from difflib import Differ
import asrtoolkit
from fuzzywuzzy import fuzz

# Load and prep statistics

In [ ]:
filenames = list(Path('predictions').glob('*.csv'))

results = pd.DataFrame()

models = list()

for idx, filename in enumerate(filenames):
    new_results = pd.read_csv(filename)
    model_label = Path(filename).stem  #.split('-')[0]
    ground_truth_label = f'{model_label}-ground_truth'
    new_results = new_results.rename(columns={'label': ground_truth_label, 
                            'prediction': model_label})
    del new_results['Unnamed: 0']
    models.append((ground_truth_label, model_label))
    new_results[ground_truth_label] = new_results.apply(lambda r: str(r[ground_truth_label]), axis=1)
    new_results[model_label] = new_results.apply(lambda r: str(r[model_label]), axis=1)
    results = pd.concat([results, new_results], axis = 1)

models = sorted(models, key=lambda b: b[1])
    
results

In [ ]:
[a[1] for a in models]

In [ ]:
NUM_STATS = 6
NUM_SUB_STATS = 5
a = ['matchratio'] * NUM_SUB_STATS + ['CER'] * NUM_SUB_STATS + ['matchratio_insensitive'] * NUM_SUB_STATS + ['CER_insensitive'] * NUM_SUB_STATS + ['matchratio_nopunc'] * NUM_SUB_STATS + ['CER_nopunc'] * NUM_SUB_STATS
b = ['mean', 'min', 'max', 'stdev', 'median'] * NUM_STATS
headers = pd.MultiIndex.from_arrays([a,b], names=('stat', 'substats'))
summary_results = pd.DataFrame(columns=headers)

def apply_stat(model_label: str, query_label: str, summary_stat_label: str, stat_function):
    results[query_label] = subset.apply(stat_function, axis=1)
    summary_results.loc[model_label, (summary_stat_label, 'mean')] = results[query_label].mean()
    summary_results.loc[model_label, (summary_stat_label, 'stdev')] = results[query_label].std()
    summary_results.loc[model_label, (summary_stat_label, 'min')] = results[query_label].min()
    summary_results.loc[model_label, (summary_stat_label, 'median')] = results[query_label].median()
    summary_results.loc[model_label, (summary_stat_label, 'max')] = results[query_label].max()
    subset[query_label] = results[query_label]
    
for ground_truth_label, model_label in models:
    subset = results[[ground_truth_label, model_label]].dropna()
    
    apply_stat(model_label, f'{model_label} CER', 'CER', lambda r: asrtoolkit.cer(str(r[ground_truth_label]), str(r[model_label])))
    apply_stat(model_label, f'{model_label} match ratio', 'matchratio', lambda r: fuzz.ratio(str(r[ground_truth_label]), str(r[model_label])))
    results[f'{model_label}-edit_distance'] = subset.apply(lambda r: distance.edit_distance(str(r[ground_truth_label]), str(r[model_label])), axis=1)
    
    subset[f'{ground_truth_label}-lower'] = subset[ground_truth_label].map(str.lower)
    subset[f'{model_label}-lower'] = subset[model_label].map(str.lower)
    apply_stat(model_label, f'{model_label} CER insensitive', 'CER_insensitive', lambda r: asrtoolkit.cer(str(r[f'{ground_truth_label}-lower']), str(r[f'{model_label}-lower'])))
    apply_stat(model_label, f'{model_label} match ratio insensitive', 'matchratio_insensitive', lambda r: fuzz.ratio(str(r[f'{ground_truth_label}-lower']), str(r[f'{model_label}-lower'])))
    
    subset[f'{ground_truth_label}-nopunc'] = subset[ground_truth_label].map(lambda s: s.translate(str.maketrans('', '', string.punctuation)))
    subset = subset.loc[subset[f'{ground_truth_label}-nopunc'].str.len() > 0]
    subset[f'{model_label}-nopunc'] = subset[model_label].map(lambda s: s.translate(str.maketrans('', '', string.punctuation)))
    subset = subset.loc[subset[f'{model_label}-nopunc'].str.len() > 0]
    apply_stat(model_label, f'{model_label} CER no punc', 'CER_nopunc', lambda r: asrtoolkit.cer(str(r[f'{ground_truth_label}-nopunc']), str(r[f'{model_label}-nopunc'])))
    apply_stat(model_label, f'{model_label} match ratio no punc', 'matchratio_nopunc',  lambda r: fuzz.ratio(str(r[f'{ground_truth_label}-lower']), str(r[f'{model_label}-lower'])))
    
    exact = results.dropna().apply(lambda r: 1 if int(r[f'{model_label}-edit_distance']) == 0 else 0, axis=1).sum()
    oboe = results.dropna().apply(lambda r: 1 if int(r[f'{model_label}-edit_distance']) == 1 else 0, axis=1).sum()
    summary_results.loc[model_label, 'exact_matches'] = pd.to_numeric(exact, downcast='integer')
    summary_results.loc[model_label, 'oboe_matches'] = pd.to_numeric(oboe, downcast='integer')
    
    err_range = stats.norm.interval(alpha=0.95, 
                                  loc=np.mean(results[f'{model_label} CER'].dropna()), 
                                  scale=stats.sem(results[f'{model_label} CER'].dropna()))
    summary_results.loc[model_label, 'cer_95_error_min'] = err_range[0]
    summary_results.loc[model_label, 'cer_95_error_max'] = err_range[1]

In [ ]:
print(summary_results[[('CER', 'mean'), ('CER_insensitive', 'mean'), ('CER_nopunc', 'mean'), 
                       ('matchratio', 'mean'), ('matchratio_insensitive', 'mean'), ('matchratio_nopunc', 'mean')]])
print(summary_results[['exact_matches', 'oboe_matches', 'cer_95_error_min', 'cer_95_error_max']])

# Generate and display graphs

In [ ]:
def plot_cer(graph):
    graph.set_title('Character Error Rate (CER)')
    graph.set_xticklabels([label[1] for label in models], rotation=15, horizontalalignment='right')
    graph.set_ylim([-1, 200])
    subset = results[[f'{label[1]} CER' for label in models]].T
    subset = np.asarray([model[1].dropna() for model in subset.iterrows()], dtype=object).T
    graph = graph.boxplot(subset)


def plot_cer_confidence_interval(graph):
    min = summary_results['cer_95_error_min']
    min = min.apply(lambda v: v if v >= 0 else 0)
    max = summary_results['cer_95_error_max']
    max = max.apply(lambda v: v if v <= 100 else 100)
    error_min = [summary_results.loc[label[1], ('CER', 'mean')] - min[label[1]] for label in models]
    error_max = [max[label[1]] - summary_results.loc[label[1], ('CER', 'mean')] for label in models]
    graph.set_title('CER 95% confidence interval')
    graph.set_xticklabels([label[1] for label in models], rotation=15, horizontalalignment='right')
    graph.set_ylim(top=100)
    graph.errorbar([label[1] for label in models], summary_results[('CER', 'mean')], yerr=[error_min, error_max], fmt='o')
    for i in range(len(models)):
        graph.text(i + 0.05, max[i] - 2, f'{max[i]:.3f}')
        graph.text(i + 0.05, min[i] + 1, f'{min[i]:.3f}')


def plot_fuzzy_match(graph):
    graph.set_title('Fuzzy Match Ratio')
    graph.set_xticklabels([label[1] for label in models], rotation=15, horizontalalignment='right')
    subset = results[[f'{label[1]} match ratio' for label in models]].T
    subset = np.asarray([model[1].dropna() for model in subset.iterrows()], dtype=object).T
    graph = graph.boxplot(subset)


def plot_close_matches(graph):  # todo
    graph.set_title('% of Exact/Almost Exact String Matches')
    exact_matches = [match/results.shape[0] for match in summary_results['exact_matches']]
    obo_matches = [match/results.shape[0] for match in summary_results['oboe_matches']]
    ex = graph.bar([label[1] for label in models], exact_matches, 
                   width=0.3, label='Exact match')
    obo = graph.bar([label[1] for label in models], obo_matches, 
                    width=0.3, bottom=exact_matches, label='Off by one')
    graph.set_xticklabels([label[1] for label in models], rotation=15, horizontalalignment='right')
    graph.set_ybound(0, 0.2)
    graph.legend()


fig, ((graph1, graph2, graph3, graph4)) = plt.subplots(4, 1, figsize=(15, 25))
# fig, graph1 = plt.subplots(1, 1, figsize=(10, 7))  # For saving individual charts


plot_cer(graph1)
plot_cer_confidence_interval(graph2)
plot_fuzzy_match(graph3)
plot_close_matches(graph4)
fig.tight_layout()

# OBO String Comparisons

In [ ]:
from Bio import pairwise2
from Bio.Seq import Seq
seq1 = Seq('foot')
seq2 = Seq('moot')
a = pairwise2.align.globalxx(seq1, seq2)
print(pairwise2.format_alignment(*a[0]))
a[0].end

In [ ]:
d = Differ()
substitution_errors = list()
deletion_errors = [dict()]
CHAR_LIST = '\' !"#&()[]*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
deletion_errors[0] = {c:0 for c in list(CHAR_LIST)}
insertion_errors = deletion_errors * 4
deletion_errors = deletion_errors * 4
    
for ground_truth_label, model_label in models:
    oboe = results[results[f'{model_label}-edit_distance'] == 1][[ground_truth_label, model_label]]
    subs = dict()
    dels = deletion_errors[len(models) - 1]
    ins = deletion_errors[len(models) - 1]
    # print(oboe)
    for idx, row in oboe.iterrows():
        differ_result = list(d.compare(row[ground_truth_label], row[model_label]))
        removed = [c[2] for c in differ_result if c[0] == '-']
        added = [c[2] for c in differ_result if c[0] == '+']
        assert len(removed) < 2 and len(added) < 2, f'Lens are wrong, -: {removed}, +: {added}'
        if len(row[ground_truth_label]) == len(row[model_label]):
            sub_chars = (removed[0], added[0])
            # print(f'sub: {sub_chars}')
            if sub_chars in subs.keys():
                subs[sub_chars] += 1
            else:
                subs[sub_chars] = 1
        elif len(row[ground_truth_label]) > len(row[model_label]):
            del_char = removed[0]
            # print(f'del: {del_char}')
            dels[del_char] += 1
        else:
            ins_char = added[0]
            # print(f'ins: {ins_char}')
            ins[ins_char] += 1
    substitution_errors.append(subs)
    deletion_errors.append(dels)
    insertion_errors.append(ins)
substitution_errors = [dict(sorted(a.items(), key=lambda item: item[1], reverse=True)) for a in substitution_errors]
# deletion_errors = [dict(sorted(a.items(), key=lambda item: item[1], reverse=True)) for a in deletion_errors]
# insertion_errors = [dict(sorted(a.items(), key=lambda item: item[1], reverse=True)) for a in insertion_errors]

In [ ]:
# print('Top Substitutions:')
# for key in list(substitution_errors[0])[:10]:
#     print(f'{key}: {substitution_errors[0][key]}')
# print('\nTop Deletions:')
# for key in list(deletion_errors[0])[:10]:
#     print(f'{key}: {deletion_errors[0][key]}')
# print('\nTop Insertions:')
# for key in list(insertion_errors[0])[:10]:
#     print(f'{key}: {insertion_errors[0][key]}')

plt.figure(figsize=(24,8))
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.bar(np.arange(len(deletion_errors[0].keys())) - 0.225, deletion_errors[0].values(), 0.45, label = models[0][1])
plt.bar(np.arange(len(deletion_errors[0].keys())) + 0.225, deletion_errors[1].values(), 0.45, label = models[1][1])
plt.xticks(np.arange(len(deletion_errors[0].keys())) , deletion_errors[0].keys())
plt.title("Deletion Errors")
plt.ylabel("Number of Occurrences")
plt.legend()
plt.show()

plt.figure(figsize=(24,8))
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.bar(np.arange(len(insertion_errors[0].keys())) - 0.225, insertion_errors[0].values(), 0.45, label = models[0][1])
plt.bar(np.arange(len(insertion_errors[0].keys())) + 0.225, insertion_errors[1].values(), 0.45, label = models[1][1])
plt.xticks(np.arange(len(insertion_errors[0].keys())) , insertion_errors[0].keys())
plt.title("Insertion Errors")
plt.ylabel("Number of Occurrences")
plt.legend()
plt.show()

In [ ]:
list_of_changes = list()
for ground_truth_label, model_label in models:
    oboes = results[results[f'{model_label}-edit_distance']==1][[f'{model_label}-ground_truth', model_label]]
    diffs = oboes.apply(lambda r: list(d.compare(r[f'{model_label}-ground_truth'], r[model_label])), axis=1)
    diffs = diffs.apply(lambda r: [a for a in r if a[0] == '+' or a[0] == '-'])
    list_of_changes.append(diffs)
print(list_of_changes[0][0:10])
missing = list()
added = list()
for changes in list_of_changes:
    missing_this_model = list()
    added_this_model = list()
    for change_list in changes:
        for change in change_list:
            if change[0] == '+':
                added_this_model.append(change[2:])
            elif change[0] == '-':
                missing_this_model.append(change[2:])
    missing.append(missing_this_model)
    added.append(added_this_model)
print(missing[0][0:10])
print(added[0][0:10])
# char_sub_summary = pd.DataFrame(columns=['Missing])

# Save these results to the filesystem

In [ ]:
results_folder = 'results'
if not os.path.exists(results_folder):
    os.makedirs(results_folder)
timestamp = datetime.now().strftime('%Y_%m_%d-%H_%M_%S')
summary_results.to_csv(Path(results_folder, f'summary-{timestamp}.csv'))
fig.savefig(Path(results_folder, f'summary-{timestamp}.png'))